In [1]:

from src.lit_model import LitModel
from pytorch_lightning import Trainer

C:\Users\trung\miniconda3\envs\image_forgery\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
model = LitModel.load_from_checkpoint("checkpoint/epoch=5-step=3786.ckpt")

C:\Users\trung\miniconda3\envs\image_forgery\lib\site-packages\lightning_fabric\utilities\cloud_io.py:57: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.


In [20]:
for p in model.model.spatial.swinv1.layers[3].parameters():
    p.requires_grad = True

In [21]:
print(sum(p.numel() for p in model.model.spatial.swinv1.layers[3].parameters() if p.requires_grad))


15366576
